In [32]:
import os
import re
import time
import csv
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import date, timedelta, datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep 
import pandas as pd
import numpy as np

In [33]:
browser = webdriver.Chrome(executable_path ='C:\Program Files (x86)\chromedriver.exe')
url = "https://seekingalpha.com/earnings/earnings-call-transcripts"
headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
browser.get(url)
html=browser.page_source


In [34]:
browser.find_element_by_xpath('//*[@id="sign-in"]').click()

In [35]:
browser.find_element_by_xpath('//*[@id="login-email"]').send_keys("phkevin41@gmail.com")
browser.find_element_by_xpath('//*[@id="login-password"]').send_keys("@DBproject8040")
browser.find_element_by_xpath('//*[@id="login-btn"]').click()

In [36]:
def addname(a,b, c): 
    Name = 'NA'
    Title = 'NA'
    Organization = 'NA'
    info = [Name, Title, Organization]
    str1 = a.split("-")
    if len(str1)==2:
        if b == "Company Participants":
            Name = str1[0].strip()
            Title = str1[1].strip()
            Organization = 'NA'
            info = [Name, Title, Organization]
        elif b == "Conference Call Participants": 
            Name = str1[0].strip()
            Organization = str1[1].strip()
            Title = "NA"
            info = [Name, Title, Organization]
    return info

In [37]:
def FillNA(colname, dataframe):
    content = dataframe
    a = 'xxxx'
    b = 'xxxx'
    content[colname].fillna('NA', inplace=True)
    for index3, x in content.iterrows():
        oldticker = x[colname]
        if a != content.loc[index3,colname] and content.loc[index3,colname] != 'NA':
            a = content.loc[index3,colname]
        content.loc[index3,colname] = a
    return content

In [38]:
def addDateTime(a_str):
    
    Ticker ='NA'
    Company_info ='NA'
    Date = 'NA'
    Time ='NA'
    parse = [Ticker, Company_info, Date, Time]
    reversed = a_str[::-1]
    replaced = reversed.replace(','[::-1], '_#_'[::-1], 1)
    a = replaced[::-1]
    splitstring = a
    outer = re.compile("\((.+)\)")
    m = outer.search(a_str)
    if(m is not None):
        test_if_title = m.group(1).split(":")
        print(m.group(1))
        print(test_if_title)
        if len(test_if_title) >1:
            Ticker = test_if_title[1]
            ar = splitstring.split("(")
            Company_info = ar[0]
            datetime = splitstring.split("_#_")
            timex = datetime[1].strip().split(" ")
            datear = datetime[0].split(" ")
            index =  len(datear)-1
            day = datear[index]
            month = datear[index-2]
            year = timex[0]
            Time = timex[1] + " " + str(timex[2]) + " " + timex[3]
            Date = month + " " + day + "," +  year
            parse = [Ticker, Company_info, Date, Time]
    return parse

In [39]:
soup = BeautifulSoup(html, 'html.parser')
page = soup.find("div", id="paging").find_all("li")
page_len = len(page)-2

k = 1
df = pd.DataFrame(columns=['Pagenum','ID','Article_Link', 'Title', 'Ticker'])
for k in range (k, page_len+1):
    browser.get('https://seekingalpha.com/earnings/earnings-call-transcripts/'+str(k))
    pagehtml = browser.page_source
    print('https://seekingalpha.com/earnings/earnings-call-transcripts/'+str(k))
    soup = BeautifulSoup(pagehtml, 'html.parser')
    try:
        articles = soup.find("ul", id="analysis-list-container").find_all("li")
        len_article = len(articles)
        i=0
        for i in range(i,len_article):
            link_article = articles[i].find("a").attrs['href']
            title_article = articles[i].find("a").get_text()
            ticker_article = articles[i].find_all("div")[0].find('span').find("a").get_text()
            df = df.append({'Pagenum':k,'ID': i, 'Article_Link':"https://seekingalpha.com/" + link_article , 'Title': title_article, 'Ticker':ticker_article }, ignore_index=True)
            i = i+1
    except:
        i=1

https://seekingalpha.com/earnings/earnings-call-transcripts/1
https://seekingalpha.com/earnings/earnings-call-transcripts/2
https://seekingalpha.com/earnings/earnings-call-transcripts/3
https://seekingalpha.com/earnings/earnings-call-transcripts/4
https://seekingalpha.com/earnings/earnings-call-transcripts/5


In [40]:
df.to_csv('transcriptpages.csv', encoding='utf-8')

In [42]:
df = pd.read_csv("LinksToTest.csv")

In [44]:
df_new = pd.DataFrame(columns=['ID','Content', 'Content_Type', 'Seq', 'SpeechSpeaker', 'QA', 'Ticker', 'Company_info', 'Date', 'Time', 'Participant_name', 'Participant_title', 'Participant_organization'])

pagelimit=0
for index, content in df.iterrows():
    pagelimit = pagelimit+1
    browser.get(content['Article_Link'])
    html_content = browser.page_source
    soup_content = BeautifulSoup(html_content, 'html.parser')
    body_content = soup_content.find("div", id="a-body")
    par = body_content.find_all("p")
    date_time_text = par[0].get_text()
    participants = body_content.find_all("p")[1].get_text()
    company_p_list = []
    itr=0
    
    for ps in par:
        try:
            parsedData= addDateTime(ps.get_text())
            x =ps.find("strong")
            if x == None:
                df_new = df_new.append({'ID': content['ID'],'Content': ps.get_text(), 'Content_Type':"Normal", 
                                        'Seq':itr, 'Ticker':parsedData[0] , 'Company_info': parsedData[1],
                                        'Date': parsedData[2], 'Time': parsedData[3]}, ignore_index=True)
            else:
                qa=ps.find("span")
                if qa==None:
                        df_new = df_new.append({'ID': content['ID'],'Content': ps.get_text(), 'Content_Type':"NotNormal", 
                                                'Seq': itr,'QA': "NA", 'Ticker':parsedData[0] , 'Company_info': parsedData[1],
                                                'Date': parsedData[2], 'Time': parsedData[3]}, ignore_index=True)
                else:
                        df_new = df_new.append({'ID': content['ID'],'Content': ps.get_text(), 'Content_Type':"NotNormal", 
                                                'Seq': itr, 'QA': qa.attrs['class'], 'Ticker':parsedData[0] , 'Company_info': parsedData[1],
                                                'Date': parsedData[2], 'Time': parsedData[3]}, ignore_index=True)

        except:
            i=1

        itr= itr+1

NYSE:NEM
['NYSE', 'NEM']
NASDAQ:MDB
['NASDAQ', 'MDB']
NYSE:BNED
['NYSE', 'BNED']
NYSE:LITB
['NYSE', 'LITB']
NYSE:IDT
['NYSE', 'IDT']
NYSE:UNP
['NYSE', 'UNP']
NASDAQ:SRPT
['NASDAQ', 'SRPT']
NASDAQ:REGN
['NASDAQ', 'REGN']
NYSE:TWTR
['NYSE', 'TWTR']
NYSE:LAIX
['NYSE', 'LAIX']


In [45]:
df_new

,ID,Content,Content_Type,Seq,SpeechSpeaker,QA,Ticker,Company_info,Date,Time,Participant_name,Participant_title,Participant_organization
0,1,Newmont Corporation (NYSE:NEM) 2021 Investor U...,Normal,0,NaN,NaN,NEM,Newmont Corporation,"December 8,2020",9:00 AM,NaN,NaN,NaN
1,1,Company Participants,NotNormal,1,NaN,NA,NA,NA,NA,NA,NaN,NaN,NaN
2,1,Eric Colby - Vice President of Investor Relati...,Normal,2,NaN,NaN,NA,NA,NA,NA,NaN,NaN,NaN
3,1,Tom Palmer - President and Chief Executive Off...,Normal,3,NaN,NaN,NA,NA,NA,NA,NaN,NaN,NaN
4,1,Rob Atkinson - Chief Operating Officer,Normal,4,NaN,NaN,NA,NA,NA,NA,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,10,[Operator Instructions]. There are no further ...,Normal,64,NaN,NaN,NA,NA,NA,NA,NaN,NaN,NaN
1247,10,Harry He,NotNormal,65,NaN,NA,NA,NA,NA,NA,NaN,NaN,NaN
1248,10,Thank you once again for joining us. If you ha...,Normal,66,NaN,NaN,NA,NA,NA,NA,NaN,NaN,NaN
1249,10,Operator,NotNormal,67,NaN,NA,NA,NA,NA,NA,NaN,NaN,NaN


In [46]:
for index, x in df_new.iterrows():
    if x.Content_Type == "NotNormal":
        partofarticle = x.Content
     
df_new['QA']=df_new['QA'].fillna('Speech')
df_new['QA']=df_new['QA'].replace(['NA'],'Header')

In [47]:
for index, x in df_new.iterrows():
    if x.Content_Type == "NotNormal":
        partofarticle = x.Content
    x.SpeechSpeaker = partofarticle

In [48]:
df_new

,ID,Content,Content_Type,Seq,SpeechSpeaker,QA,Ticker,Company_info,Date,Time,Participant_name,Participant_title,Participant_organization
0,1,Newmont Corporation (NYSE:NEM) 2021 Investor U...,Normal,0,Operator,Speech,NEM,Newmont Corporation,"December 8,2020",9:00 AM,NaN,NaN,NaN
1,1,Company Participants,NotNormal,1,Company Participants,Header,NA,NA,NA,NA,NaN,NaN,NaN
2,1,Eric Colby - Vice President of Investor Relati...,Normal,2,Company Participants,Speech,NA,NA,NA,NA,NaN,NaN,NaN
3,1,Tom Palmer - President and Chief Executive Off...,Normal,3,Company Participants,Speech,NA,NA,NA,NA,NaN,NaN,NaN
4,1,Rob Atkinson - Chief Operating Officer,Normal,4,Company Participants,Speech,NA,NA,NA,NA,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,10,[Operator Instructions]. There are no further ...,Normal,64,Operator,Speech,NA,NA,NA,NA,NaN,NaN,NaN
1247,10,Harry He,NotNormal,65,Harry He,Header,NA,NA,NA,NA,NaN,NaN,NaN
1248,10,Thank you once again for joining us. If you ha...,Normal,66,Harry He,Speech,NA,NA,NA,NA,NaN,NaN,NaN
1249,10,Operator,NotNormal,67,Operator,Header,NA,NA,NA,NA,NaN,NaN,NaN


In [49]:
for index2, x in df_new.iterrows():
    Name_Info = addname(x.Content, x.SpeechSpeaker, x.Company_info)
    df_new.loc[df_new.index == index2,'Participant_name']= Name_Info[0]
    df_new.loc[df_new.index == index2,'Participant_title']= Name_Info[1]
    df_new.loc[df_new.index == index2,'Participant_organization']= Name_Info[2]

In [50]:
df_new

,ID,Content,Content_Type,Seq,SpeechSpeaker,QA,Ticker,Company_info,Date,Time,Participant_name,Participant_title,Participant_organization
0,1,Newmont Corporation (NYSE:NEM) 2021 Investor U...,Normal,0,Operator,Speech,NEM,Newmont Corporation,"December 8,2020",9:00 AM,NA,NA,NA
1,1,Company Participants,NotNormal,1,Company Participants,Header,NA,NA,NA,NA,NA,NA,NA
2,1,Eric Colby - Vice President of Investor Relati...,Normal,2,Company Participants,Speech,NA,NA,NA,NA,Eric Colby,Vice President of Investor Relations and Globa...,NA
3,1,Tom Palmer - President and Chief Executive Off...,Normal,3,Company Participants,Speech,NA,NA,NA,NA,Tom Palmer,President and Chief Executive Officer,NA
4,1,Rob Atkinson - Chief Operating Officer,Normal,4,Company Participants,Speech,NA,NA,NA,NA,Rob Atkinson,Chief Operating Officer,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,10,[Operator Instructions]. There are no further ...,Normal,64,Operator,Speech,NA,NA,NA,NA,NA,NA,NA
1247,10,Harry He,NotNormal,65,Harry He,Header,NA,NA,NA,NA,NA,NA,NA
1248,10,Thank you once again for joining us. If you ha...,Normal,66,Harry He,Speech,NA,NA,NA,NA,NA,NA,NA
1249,10,Operator,NotNormal,67,Operator,Header,NA,NA,NA,NA,NA,NA,NA


In [51]:
df1 = FillNA('Ticker',df_new)
df2 = FillNA('Company_info',df1)
df3 = FillNA('Date',df2)
df4 = FillNA('Time',df3)


In [52]:
df4

,ID,Content,Content_Type,Seq,SpeechSpeaker,QA,Ticker,Company_info,Date,Time,Participant_name,Participant_title,Participant_organization
0,1,Newmont Corporation (NYSE:NEM) 2021 Investor U...,Normal,0,Operator,Speech,NEM,Newmont Corporation,"December 8,2020",9:00 AM,NA,NA,NA
1,1,Company Participants,NotNormal,1,Company Participants,Header,NEM,Newmont Corporation,"December 8,2020",9:00 AM,NA,NA,NA
2,1,Eric Colby - Vice President of Investor Relati...,Normal,2,Company Participants,Speech,NEM,Newmont Corporation,"December 8,2020",9:00 AM,Eric Colby,Vice President of Investor Relations and Globa...,NA
3,1,Tom Palmer - President and Chief Executive Off...,Normal,3,Company Participants,Speech,NEM,Newmont Corporation,"December 8,2020",9:00 AM,Tom Palmer,President and Chief Executive Officer,NA
4,1,Rob Atkinson - Chief Operating Officer,Normal,4,Company Participants,Speech,NEM,Newmont Corporation,"December 8,2020",9:00 AM,Rob Atkinson,Chief Operating Officer,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,10,[Operator Instructions]. There are no further ...,Normal,64,Operator,Speech,LAIX,LAIX Inc.,"Call 26,2020",8:00 PM,NA,NA,NA
1247,10,Harry He,NotNormal,65,Harry He,Header,LAIX,LAIX Inc.,"Call 26,2020",8:00 PM,NA,NA,NA
1248,10,Thank you once again for joining us. If you ha...,Normal,66,Harry He,Speech,LAIX,LAIX Inc.,"Call 26,2020",8:00 PM,NA,NA,NA
1249,10,Operator,NotNormal,67,Operator,Header,LAIX,LAIX Inc.,"Call 26,2020",8:00 PM,NA,NA,NA


In [31]:
df4.to_csv("8040DataSource.csv")